In [319]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
import numpy as np

# Load the dataset
gamedf = pd.read_csv(r"C:\Users\Zain\Downloads\GameRecommender-main\GameRecommender-main\vgsales.csv")
temp_gamedf = pd.read_csv(r"C:\Users\Zain\Downloads\GameRecommender-main\GameRecommender-main\vgsales.csv")

# Label encoding for string columns
label_encoder = LabelEncoder()
string_columns = gamedf.select_dtypes(include=['object']).columns
string_columns = string_columns.drop('Name')
for column in string_columns:
    temp_gamedf[column] = label_encoder.fit_transform(temp_gamedf[column])
print(temp_gamedf.head())

   Rank                      Name  Platform    Year  Genre  Publisher  \
0     0                Wii Sports        26  2006.0     10        359   
1     1         Super Mario Bros.        11  1985.0      4        359   
2     2            Mario Kart Wii        26  2008.0      6        359   
3     3         Wii Sports Resort        26  2009.0     10        359   
4     4  Pokemon Red/Pokemon Blue         5  1996.0      7        359   

   NA_Sales  EU_Sales  JP_Sales  Other_Sales  Global_Sales  
0     41.49     29.02      3.77         8.46         82.74  
1     29.08      3.58      6.81         0.77         40.24  
2     15.85     12.88      3.79         3.31         35.82  
3     15.75     11.01      3.28         2.96         33.00  
4     11.27      8.89     10.22         1.00         31.37  


In [320]:
#scaling for numerical columns
numerical_columns = temp_gamedf.select_dtypes(include=['float64', 'int64', 'int32']).columns
numerical_columns = numerical_columns.drop('Rank')
scaler = MinMaxScaler()
temp_gamedf[numerical_columns] = scaler.fit_transform(temp_gamedf[numerical_columns])
print(temp_gamedf.head())

   Rank                      Name  Platform   Year     Genre  Publisher  \
0     0                Wii Sports  0.866667  0.650  0.909091   0.621107   
1     1         Super Mario Bros.  0.366667  0.125  0.363636   0.621107   
2     2            Mario Kart Wii  0.866667  0.700  0.545455   0.621107   
3     3         Wii Sports Resort  0.866667  0.725  0.909091   0.621107   
4     4  Pokemon Red/Pokemon Blue  0.166667  0.400  0.636364   0.621107   

   NA_Sales  EU_Sales  JP_Sales  Other_Sales  Global_Sales  
0  1.000000  1.000000  0.368885     0.800378      1.000000  
1  0.700892  0.123363  0.666341     0.072848      0.486281  
2  0.382020  0.443832  0.370841     0.313150      0.432854  
3  0.379610  0.379394  0.320939     0.280038      0.398767  
4  0.271632  0.306340  1.000000     0.094607      0.379064  


In [321]:
from sklearn.feature_selection import SelectKBest, f_classif
import pandas as pd
import numpy as np
X = temp_gamedf.drop('Name', axis=1).dropna()
y = temp_gamedf['Name'].dropna()
print(X.describe())
common_indices = X.index.intersection(y.index)

X = X.loc[common_indices]
y = y.loc[common_indices]

X = X.dropna()

selector = SelectKBest(score_func=f_classif, k=5)

X_selected = selector.fit_transform(X, y)

selected_feature_indices = selector.get_support(indices=True)

selected_feature_names = X.columns[selected_feature_indices]
print(selected_feature_names)


               Rank      Platform          Year         Genre     Publisher  \
count  16327.000000  16327.000000  16327.000000  16327.000000  16327.000000   
mean    8291.868194      0.526488      0.660161      0.447814      0.506908   
std     4792.669778      0.279071      0.145725      0.341893      0.306924   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%     4135.500000      0.233333      0.575000      0.090909      0.237024   
50%     8294.000000      0.533333      0.675000      0.454545      0.558824   
75%    12440.500000      0.700000      0.750000      0.727273      0.788927   
max    16599.000000      1.000000      1.000000      1.000000      1.000000   

           NA_Sales      EU_Sales      JP_Sales   Other_Sales  Global_Sales  
count  16327.000000  16327.000000  16327.000000  16327.000000  16327.000000  
mean       0.006397      0.005085      0.007697      0.004572      0.006409  
std        0.019802      0.017532      0.030485      0

In [322]:
from sklearn.metrics import r2_score

def Similarity(n1, temp_gamedf, selected_features):
    print('Similarity Checkpoint 1')
    X_train = temp_gamedf[selected_features].dropna()
    y_train = temp_gamedf.loc[X_train.index, 'Rank']

    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    knn_model = KNeighborsRegressor(n_neighbors=5)
    lr_model = LinearRegression()
    print('X_train')
    print(X_train)
    print('Similarity Checkpoint 2')
    rf_model.fit(X_train.values, y_train.values)
    knn_model.fit(X_train.values, y_train.values)
    lr_model.fit(X_train.values, y_train.values)
    print('Similarity Checkpoint 3')

    print('n1')
    print(n1)

    print('Similarity Checkpoint 5')
    rf_predictions = rf_model.predict(n1)
    knn_predictions = knn_model.predict(n1)
    lr_predictions = lr_model.predict(n1)

    print('Similarity Checkpoint 6')
    rf_similarity = np.abs(rf_predictions - rf_model.predict(X_train.values))
    #knn_similarity = np.abs(knn_predictions - knn_model.predict(X_train.values))
    #lr_similarity = np.abs(lr_predictions - lr_model.predict(X_train.values))

    print('Similarity Checkpoint 7')
    max_similarity = np.max(np.vstack([rf_similarity]), axis=0)
    rf_r2 = r2_score(y_train, rf_model.predict(X_train.values))
    knn_r2 = r2_score(y_train, knn_model.predict(X_train.values))
    lr_r2 = r2_score(y_train, lr_model.predict(X_train.values))

    print(f'Random Forest R-squared: {rf_r2}')
    print(f'KNN R-squared: {knn_r2}')
    print(f'Linear Regression R-squared: {lr_r2}')
    return max_similarity


In [323]:
def getNeighbors(basegame, K, temp_gamedf, selected_features):
    print('getNeighbors Checkpoint 1')
    basegame_features = basegame[selected_features].values.reshape(1, -1)
    all_game_features = temp_gamedf[selected_features].values

    nan_mask = np.isnan(all_game_features.astype(float)).any(axis=1)
    all_game_features = all_game_features[~nan_mask]
    all_game_ranks = temp_gamedf.loc[~nan_mask, 'Rank']

    print('getNeighbors Checkpoint 2')
    similarities = np.abs(Similarity(basegame_features, temp_gamedf, selected_features))

    print('getNeighbors Checkpoint 3')
    neighbor_indices = np.argsort(similarities)[:K]

    print('getNeighbors Checkpoint 4')
    neighbors = [(all_game_ranks.iloc[i], similarities[i]) for i in neighbor_indices]

    return neighbors

In [324]:
# Function to predict scores
def predict_score(temp_gamedf, selected_features):
    name = 'Call of Duty'
    matching_games = temp_gamedf[temp_gamedf['Name'].str.contains(name)]

    if matching_games.empty:
        print(f"No games found with a name containing '{name}'.")
        return

    new_game = matching_games.iloc[0].to_frame().T
    print('Selected Game:', new_game['Name'].values[0])

    K = 20
    neighbors = getNeighbors(new_game, K, temp_gamedf, selected_features)
    print('\nRecommended Games:\n')
    print(neighbors)
    for neighbor_info in neighbors:
        neighbor_rank= neighbor_info
        neighbor_game = gamedf[gamedf['Rank'] == neighbor_rank[0]]
        if not neighbor_game.empty:
            if neighbor_game.iloc[0]['Name'] != new_game.iloc[0]['Name']:
                print(neighbor_game.iloc[0]['Name'])


In [325]:
predict_score(temp_gamedf, selected_feature_names)

Selected Game: Call of Duty: Modern Warfare 3
getNeighbors Checkpoint 1
getNeighbors Checkpoint 2
Similarity Checkpoint 1
X_train
        Year     Genre  Publisher  JP_Sales  Global_Sales
0      0.650  0.909091   0.621107  0.368885      1.000000
1      0.125  0.363636   0.621107  0.666341      0.486281
2      0.700  0.545455   0.621107  0.370841      0.432854
3      0.725  0.909091   0.621107  0.320939      0.398767
4      0.400  0.636364   0.621107  1.000000      0.379064
...      ...       ...        ...       ...           ...
16593  0.550  0.363636   0.465398  0.000000      0.000000
16594  0.575  0.727273   0.416955  0.000000      0.000000
16595  0.700  0.545455   0.036332  0.000000      0.000000
16596  0.750  0.454545   0.013841  0.000000      0.000000
16597  0.575  0.363636   0.944637  0.000000      0.000000

[16327 rows x 5 columns]
Similarity Checkpoint 2
Similarity Checkpoint 3
n1
[[0.7750000000000057 0.7272727272727273 0.03633217993079585
  0.012720156555772993 0.178290825577